In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import json
import folium

from sklearn import preprocessing 

In [49]:
df = pd.read_csv('./자치구별 공원 현황.csv')

In [20]:
df.head()

,지역,공원면적,공원수,인구,면적,면적비율,인당면적
0,강남구,3626398,7,544055,39500000.0,9.180754,6.665499
1,강동구,1526390,7,463998,24590000.0,6.207361,3.289648
2,강북구,3714902,4,311569,23600000.0,15.741110,11.923208
3,강서구,2194154,9,585901,41440000.0,5.294773,3.744923
4,관악구,10431453,2,509803,29570000.0,35.277149,20.461733


In [21]:
df2 = df.groupby('지역').공원면적.sum().to_frame()
df2.head()

,공원면적
지역,
강남구,3626398
강동구,1526390
강북구,3714902
강서구,2194154
관악구,10431453


In [22]:
df3 = df.groupby('지역').인구.sum().to_frame()
df3.head()

,인구
지역,
강남구,544055
강동구,463998
강북구,311569
강서구,585901
관악구,509803


In [26]:
df4 = df.groupby('지역').면적.sum().to_frame()
df4.head()

,면적
지역,
강남구,39500000.0
강동구,24590000.0
강북구,23600000.0
강서구,41440000.0
관악구,29570000.0


In [28]:
df_total = pd.concat([df2,df3,df4], axis=1)
df_total.head()

,공원면적,인구,면적
지역,,,
강남구,3626398,544055,39500000.0
강동구,1526390,463998,24590000.0
강북구,3714902,311569,23600000.0
강서구,2194154,585901,41440000.0
관악구,10431453,509803,29570000.0


In [29]:
df_total['면적대비공원면적'] = df_total['공원면적'] / df_total['면적']
df_total.head()

,공원면적,인구,면적,면적대비공원면적
지역,,,,
강남구,3626398,544055,39500000.0,0.091808
강동구,1526390,463998,24590000.0,0.062074
강북구,3714902,311569,23600000.0,0.157411
강서구,2194154,585901,41440000.0,0.052948
관악구,10431453,509803,29570000.0,0.352771


In [30]:
df_total['인구대비공원면적'] = df_total['공원면적'] / df_total['인구']
df_total.head()

,공원면적,인구,면적,면적대비공원면적,인구대비공원면적
지역,,,,,
강남구,3626398,544055,39500000.0,0.091808,6.665499
강동구,1526390,463998,24590000.0,0.062074,3.289648
강북구,3714902,311569,23600000.0,0.157411,11.923208
강서구,2194154,585901,41440000.0,0.052948,3.744923
관악구,10431453,509803,29570000.0,0.352771,20.461733


In [34]:
area_criteria = [-1, 100000, 1000000, 12000000]
label = ["소형", "중형", "대형"]
size_info = [3, 7, 15]

In [35]:
area = df_total['공원면적']

In [37]:
scale = pd.cut(area, area_criteria, labels=label)
scale

지역
강남구      대형
강동구      대형
강북구      대형
강서구      대형
관악구      대형
광진구      중형
구로구      대형
금천구      소형
노원구     NaN
도봉구      대형
동대문구     중형
동작구      대형
마포구      대형
서대문구     대형
서초구     NaN
성동구      중형
성북구      대형
송파구      대형
양천구      대형
영등포구     중형
용산구      중형
은평구      대형
종로구      대형
중구       대형
중랑구      대형
Name: 공원면적, dtype: category
Categories (3, object): ['소형' < '중형' < '대형']

In [38]:
size = pd.cut(area, area_criteria, labels=size_info)

In [40]:
df_total2 = pd.concat([df_total, scale, size], axis=1)
df_total2.head()

,공원면적,인구,면적,면적대비공원면적,인구대비공원면적,공원면적,공원면적
지역,,,,,,,
강남구,3626398,544055,39500000.0,0.091808,6.665499,대형,15
강동구,1526390,463998,24590000.0,0.062074,3.289648,대형,15
강북구,3714902,311569,23600000.0,0.157411,11.923208,대형,15
강서구,2194154,585901,41440000.0,0.052948,3.744923,대형,15
관악구,10431453,509803,29570000.0,0.352771,20.461733,대형,15


In [42]:
df_total2.columns = ['공원면적', '인구', '면적', '면적대비공원면적', '인구대비공원면적', '분류', '크기']
df_total2.head()

,공원면적,인구,면적,면적대비공원면적,인구대비공원면적,분류,크기
지역,,,,,,,
강남구,3626398,544055,39500000.0,0.091808,6.665499,대형,15
강동구,1526390,463998,24590000.0,0.062074,3.289648,대형,15
강북구,3714902,311569,23600000.0,0.157411,11.923208,대형,15
강서구,2194154,585901,41440000.0,0.052948,3.744923,대형,15
관악구,10431453,509803,29570000.0,0.352771,20.461733,대형,15


In [32]:
df_loc = pd.read_csv('./서울시 주요 공원현황.csv', encoding='EUC-KR')
df_loc = df_loc[['공원명', '지역', 'X좌표(WGS84)', 'Y좌표(WGS84)']]
df_loc.columns = ['공원명', '지역', '경도', '위도']
df_loc.head()

,공원명,지역,경도,위도
0,남산도시자연공원,중구,126.990377,37.550140
1,길동생태공원,강동구,127.154779,37.540394
2,서울대공원,과천시,127.019846,37.426449
3,서울숲,성동구,127.041798,37.543072
4,월드컵공원,마포구,126.878907,37.571805


In [ ]:
# df_gu = df_loc.groupby('구')[['인구']].sum()
# df_gu.reset_index(inplace=True)
# df_gu.head()

In [ ]:
df_loc.groupby('지역').

In [33]:
map = folium.Map(location=[37.581, 126.986], zoom_start=11)

for i in df.index:
    folium.CircleMarker([df_loc.위도[i], df_loc.경도[i]],
                        radius = int(df.크기[i]),
                        tooltip=f"{df.공원명[i]}({df.면적[i]: ,d})m2",
                        color = 'crimson',
                        fill_color = 'crimson'
                         ).add_to(map)

# title = '<h3 align="center" style="font-size:20px">서울공원</h3>'
# map.get_root().html.add_child(folium.Element(title))

map

AttributeError: ignored

In [ ]:
# solution

In [44]:
filename = './seoul-gu-simple.json'    # 구 경계 데이터
with open(filename, encoding = 'utf-8') as json_file:
  geo_data = json.load(json_file)

In [48]:
geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '강동구',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.16683184366129, 37.57672487388627],
      [127.18408792330152, 37.55814280369575],
      [127.16530984307447, 37.54221851258693],
      [127.14672806823502, 37.51415680680291],
      [127.12123165719615, 37.52528270089],
      [127.1116764203608, 37.540669955324965],
      [127.11519584981606, 37.557533180704915]]]}},
  {'type': 'Feature',
   'id': '송파구',
   'properties': {'code': '11240',
    'name': '송파구',
    'name_eng': 'Songpa-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.0690698130372, 37.522279423505026],
      [127.10087519791962, 37.524841220167055],
      [127.1116764203608, 37.540669955324965],
      [127.12123165719615, 37.52528270089

In [51]:
center = [37.581, 126.986]
map = folium.Map(location=center, zoom_start=11,
                 tiles='Stamen Toner')
folium.Choropleth(
                   geo_data = geo_data, data=df,
                  columns=('지역', '공원면적'),
                  key_on = 'feature.id',
                  fill_color = 'BuPu',
                  legend_name = '공원면적'
).add_to(map)

map

In [56]:
df9 = pd.read_csv('서울공원요약.csv')
df9

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683,소형,3
2,서울대공원,과천시,127.019846,37.426449,9132690,대형,15
3,서울숲,성동구,127.041798,37.543072,480994,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085,대형,15
...,...,...,...,...,...,...,...
125,숭인근린공원,종로구,127.017827,37.576442,45615,소형,3
126,서일대뒷산공원,중랑구,127.098782,37.587716,631021,중형,7
127,용마도시자연공원(사가정공원),중랑구,127.095997,37.579762,109635,중형,7
128,문화비축기지,마포구,126.893246,37.571718,140022,중형,7


In [58]:
park =df9

In [59]:
park.head()

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683,소형,3
2,서울대공원,과천시,127.019846,37.426449,9132690,대형,15
3,서울숲,성동구,127.041798,37.543072,480994,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085,대형,15


In [60]:
center = [37.581, 126.986]
map = folium.Map(location=center, zoom_start=11,
                 tiles='Stamen Toner')
folium.Choropleth(
                   geo_data = geo_data, data=df,
                  columns=('지역', '공원면적'),
                  key_on = 'feature.id',
                  fill_color = 'BuPu',
                  legend_name = '공원면적'
).add_to(map)


for i in park.index:
    folium.CircleMarker([park.위도[i], park.경도[i]],
                         radius=int(park.크기[i]),
                         tooltip = f'{park.공원명[i]}({park.면적[i]:,d})',
                         color='green', fill_color='green'
                         
                         
                         ).add_to(map)

map

In [ ]:
center = [37.581, 126.986]
map = folium.Map(location=center, zoom_start=11,
                 tiles='Stamen Toner')
folium.Choropleth(
                   geo_data = geo_data, data=df,
                  columns=('지역', '공원면적'),
                  key_on = 'feature.id',
                  fill_color = 'BuPu',
                  legend_name = '공원면적'
).add_to(map)


for i in park.index:
    folium.CircleMarker([park.위도[i], park.경도[i]],
                         radius=int(park.크기[i]),
                         tooltip = f'{park.공원명[i]}({park.면적[i]:,d})',
                         color='green', fill_color='green'
                         
                         
                         ).add_to(map)

map